In [9]:
import os
import json
import minsearch
from openai import OpenAI
from tqdm.auto import tqdm
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

In [10]:
load_dotenv()

# get open api key from .env file
open_ai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = open_ai_api_key)

In [11]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [12]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [13]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [14]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [15]:
q = 'the course has already started, can I still enroll?'

In [16]:
index.fit(documents)

In [17]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It depends on the policies of the institution or platform offering the course. Many universities and online learning platforms have specific guidelines regarding late enrollment. Here are some steps you can take:\n\n1. **Check the Course Page**: Look for any information on the course page about late enrollment.\n\n2. **Contact the Instructor**: Reach out to the course instructor or professor to inquire if they would allow a late enrollment.\n\n3. **Contact Student Services or Registrar**: If it's a university course, the registrar's office or student services department can usually provide guidance on late enrollment procedures.\n\n4. **Online Learning Platforms**: For platforms like Coursera, Udacity, or edX, check the FAQ section or contact their support team.\n\n5. **Catch-Up Plan**: Be prepared to discuss how you plan to catch up on missed material if late enrollment is allowed.\n\nIf late enrollment is permitted, make sure to understand any requirements or deadlines for making up

In [18]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )

    return results

def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT: {context}
     """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question = query, context = context).strip()

    return prompt

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = [{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [19]:
query = 'how do I run kafka?'

rag(query)

'To run Kafka in Java, follow these steps:\n\n1. **Navigate to your project directory.**\n\n2. **Run the following command in the terminal:**\n\n   ```bash\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\nReplace `<jar_name>` with the name of the compiled JAR file you\'re working with.\n\nIf you\'re working with Kafka in Python and encounter a "Module `kafka` not found" error, you should:\n\n1. **Create a virtual environment:**\n\n   ```bash\n   python -m venv env\n   ```\n\n2. **Activate the virtual environment:**\n\n   - On MacOS or Linux:\n\n     ```bash\n     source env/bin/activate\n     ```\n\n   - On Windows:\n\n     ```bash\n     env\\Scripts\\activate\n     ```\n\n3. **Install the necessary packages:**\n\n   ```bash\n   pip install -r ../requirements.txt\n   ```\n\n4. **Ensure Docker images are up and running if required.**\n\nTo resolve any potential permissions issues when running scripts, such as `build.sh`, you 

In [20]:
rag('the course has already started, can I still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [21]:
es_client = Elasticsearch('http://localhost:9200')

In [22]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index = index_name, body = index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/8EOToRGkSQi5jpdt7R_gSA] already exists')

In [23]:
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [25]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [24]:
query = 'I just disovered the course. Can I still join it?'

In [27]:
search_results = elastic_search(query)
search_results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [28]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [29]:
rag(query)

"Yes, you can still join the course even if it has already started. You are eligible to submit homework and participate in the course activities. However, keep in mind that there are deadlines for turning in the final projects, so it's important not to leave everything until the last minute."